In [ ]:
import boto3

In [ ]:
aws_region = 'us-east-1'
repository_name = 'esmaeil_gpt2_image'
image_tag = 'latest'

In [ ]:
account_id = boto3.client('sts').get_caller_identity()['Account']
ecr_client = boto3.client('ecr', region_name=aws_region)

In [ ]:
# 1. Create repo (if doesn't exist)
# aws ecr create-repository --repository-name my-sagemaker-image

try:
    ecr_client.create_repository(repositoryName=repository_name)
except ecr_client.exceptions.RepositoryAlreadyExistsException:
    pass

In [ ]:
# 2. Get login command
# aws ecr get-login-password --region us-west-2
auth_token = ecr_client.get_authorization_token()
username, password = auth_token['authorizationData'][0]['authorizationToken'].split(':')
ecr_url = auth_token['authorizationData'][0]['proxyEndpoint']

In [ ]:
# 3. Login to Docker
!docker login --u AWS --p {password} {ecr_url}


In [ ]:
# 4. Build Docker image
image_uri = f'{account_id}.dkr.ecr.{aws_region}.amazonaws.com/{repository_name}:{image_tag}'
!docker build -t {repository_name} .
!docker tag {repository_name} {image_uri}

In [ ]:
# 5. Push to ECR
!docker push {image_uri}

In [ ]:
# 6. Ready to use `image_uri` in SageMaker Estimator
print(f'Docker image pushed to: {image_uri}')